In [1]:
from tuning.utils.utils import chat_template_func
from vllm import LLM, SamplingParams
from tuning.config import MODELS_DIR
from tuning.inference.config_inference import VLLMSamplingParamsConfig
from tuning.data.test_dataset import get_ifeval_test_dataset
import os
from tqdm import tqdm

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 02-04 23:26:46 [__init__.py:244] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This m

/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning


In [2]:
model_name = "llama3-1B"
model_path = os.path.join(MODELS_DIR, model_name)
llm = LLM(model=model_path)

INFO 02-04 23:27:06 [config.py:841] This model supports multiple tasks: {'classify', 'reward', 'generate', 'embed'}. Defaulting to 'generate'.


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 02-04 23:27:06 [config.py:3368] Downcasting torch.float32 to torch.bfloat16.
INFO 02-04 23:27:06 [config.py:1472] Using max model len 131072
INFO 02-04 23:27:08 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 02-04 23:27:09 [__init__.py:2662] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 02-04 23:27:14 [__init__.py:244] Automatically detected platform cuda.
INFO 02-04 23:27:16 [core.py:526] Waiting for init message from front-end.
INFO 02-04 23:27:16 [core.py:69] Initializing a V1 LLM engine (v0.9.2) with config: model='/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B', speculative_config=None, tokenizer='/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B', skip_tokenizer_init=Fals

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.18it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.18it/s]



INFO 02-04 23:27:18 [default_loader.py:272] Loading weights took 0.90 seconds
INFO 02-04 23:27:18 [gpu_model_runner.py:1801] Model loading took 2.3205 GiB and 1.083872 seconds
INFO 02-04 23:27:22 [backends.py:508] Using cache directory: /home/shougan/.cache/vllm/torch_compile_cache/ab3adac7a0/rank_0_0/backbone for vLLM's torch.compile
INFO 02-04 23:27:22 [backends.py:519] Dynamo bytecode transform time: 3.73 s
INFO 02-04 23:27:25 [backends.py:155] Directly load the compiled graph(s) for shape None from the cache, took 2.305 s
INFO 02-04 23:27:26 [monitor.py:34] torch.compile takes 3.73 s in total
INFO 02-04 23:27:26 [gpu_worker.py:232] Available KV cache memory: 64.03 GiB
INFO 02-04 23:27:26 [kv_cache_utils.py:716] GPU KV cache size: 2,097,984 tokens
INFO 02-04 23:27:26 [kv_cache_utils.py:720] Maximum concurrency for 131,072 tokens per request: 16.01x


Capturing CUDA graph shapes: 100%|█████████████| 67/67 [00:15<00:00,  4.20it/s]


INFO 02-04 23:27:43 [gpu_model_runner.py:2326] Graph capturing finished in 16 secs, took 0.43 GiB
INFO 02-04 23:27:43 [core.py:172] init engine (profile, create kv cache, warmup model) took 24.60 seconds


In [11]:
config = VLLMSamplingParamsConfig()
config.n = 1
sampling_params = SamplingParams(**config.model_dump())
print(sampling_params)

SamplingParams(n=1, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=0.5, top_p=0.9, top_k=150, min_p=0.0, seed=None, stop=['<|im_end|>', '<|end_of_text|>'], stop_token_ids=[], bad_words=[], include_stop_str_in_output=False, ignore_eos=False, max_tokens=4096, min_tokens=0, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True, truncate_prompt_tokens=None, guided_decoding=None, extra_args=None)


In [4]:
tokenizer = llm.get_tokenizer()
tokenizer = chat_template_func(tokenizer)   
chat_template = tokenizer.chat_template

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


In [5]:
dataset = get_ifeval_test_dataset()

In [13]:
outputs = llm.chat(
    dataset["messages"][:5],  # Just run on first 10 samples for demo
    sampling_params, 
    chat_template=chat_template,
)

Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

KeyboardInterrupt: 